LangChainMemory
```
이전 대화 내용을 기억해서 문맥을 유지하는 역할 LangChain 0.3x 부터는 LCEL 기반으로 체인을 구성, RunnableWithMessageHistory, ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결
```

In [8]:
# 환경설정
%pip install --quiet langchain-openai langchain-community python-dotenv langchain_redis

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_core.chat_history import InMemoryChatMessageHistory # 인메모리 -> 어플리케이션 자체에 저장, 외부 불가능 #블로그 
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")


human: 안녕하세요 제 이름은 홍길동 입니다.
ai: 안녕하세요 홍길동님, 무엇을 도와드릴까요?


In [ ]:
# # Radis 기반  채팅 기록 저장소
# from langchain_redis import RedisChatMessageHistory
# import os
# REDIS_URL =os.getenv("REDIS_URL","redis//localhost:6379") # 환경변수에서 REDIS_URL을 가져오고, 없으면 기본값 사용
# session_id = "user_123" # 세션 ID 설정
# history = RedisChatMessageHistory(
#     session_id=session_id,
#     redis_url=REDIS_URL)
# history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
# history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# # 현재까지의 대화 내용 확인
# for msg in history.messages:
#     print(f"{msg.type}: {msg.content}")


<function os.getenv(key, default=None)>

In [10]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중 사용자 챗봇
# 핵심 : session_id를 키로하는 메모리 저장소 만들고 사용자의 대환느 키별로 저장한다.
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뛰어난 한국어 상담 챗봇입니다 질문에 친절하고 자세히 답변해주세요."),
     # history 키로 전달된 메세지 목록은 체인 실행 시 해당 위치에 넣겠다는 의미 
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)



In [11]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [ ]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_session_history를 구현

In [17]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 전체 매핑 
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """세션 ID에 해당하는 대화 기록을 반환합니다. 존재하지 않으면 새로 생성합니다."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    output_messages_key="history",
)

In [18]:
# 두개의 세션을 번갈아가며 대화 RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다. 
sessions = ['user_a', 'user_b']
questions = [
    "안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?", # user_a의 첫번째 질문
    "저는 김철수입니다. 당신은 어떤 일을 하시나요?", # user_b의 첫번째 질문
    "저는 프로그래머입니다. 당신은 어떤 일을 하시나요?",# user_a의 두번째 질문
    "저는 디자이너입니다. 요즘 어떤 프로젝트를 하고 계신가요?" # user_b의 두번째 질문
]
for i, question in enumerate(questions):
    session_id = sessions[i % 2]  # 세션 ID를 번갈아가며 사용
    result = chatbot.invoke({'input': question},config={'configurable': {'session_id': session_id}})
    print(f"Session {session_id} 질문: {question}")
    print(f"Session {session_id} 챗봇: {result}\n")


KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'input'] Received: ['input']\nNote: if you intended {chat_history} to be part of the string and not a variable, please escape it with double curly braces like: '{{chat_history}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "